##Disclaimer


Chatgpt was involved


prompts:
how to concatenate features for example HOG with LBP

how to make color histogram for additional feature



In [1]:
%pip install opencv-python numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.metrics import accuracy_score

## Using CIFAR-10

In [3]:
kernel_sharpen = np.array([[-1, -1, -1],
                    [-1, 8, -1],
                    [-1, -1, -1]])
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        images = dict[b'data']
        dict[b'data_processed']=[]
        for i, image in enumerate(images):
            image = image.reshape(3, 32, 32).transpose(1,2,0).astype("uint8")

            # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # # Log transform
            # image = (image)/255
            # c = 255 / (1+np.max(image))
            # image = c*np.log(1+image)
            # image = np.array(image, dtype=np.uint8)

            # img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

            # # equalize the histogram of the Y channel
            # img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

            # # convert the YUV image back to RGB format
            # image = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
            # image = abs(image+cv2.filter2D(image,-1,kernel_sharpen))
            
            dict[b'data_processed'].append(image)
        dict[b'data_processed']=np.array(dict[b'data_processed'])

    return dict

In [4]:
first = unpickle("cifar-10-batches-py/data_batch_1")
second = unpickle("cifar-10-batches-py/data_batch_2")
third = unpickle("cifar-10-batches-py/data_batch_3")
fourth = unpickle("cifar-10-batches-py/data_batch_4")
fifth = unpickle("cifar-10-batches-py/data_batch_5")

DICT_TRAIN = first
DICT_TRAIN[b'data_processed']=np.append(DICT_TRAIN[b'data_processed'], second[b'data_processed'],0)
DICT_TRAIN[b'labels']=np.append(DICT_TRAIN[b'labels'], second[b'labels'],0)

DICT_TRAIN[b'data_processed']=np.append(DICT_TRAIN[b'data_processed'], third[b'data_processed'],0)
DICT_TRAIN[b'labels']=np.append(DICT_TRAIN[b'labels'], third[b'labels'],0)

DICT_TRAIN[b'data_processed']=np.append(DICT_TRAIN[b'data_processed'], fourth[b'data_processed'],0)
DICT_TRAIN[b'labels']=np.append(DICT_TRAIN[b'labels'], fourth[b'labels'],0)

DICT_TRAIN[b'data_processed']=np.append(DICT_TRAIN[b'data_processed'], fifth[b'data_processed'],0)
DICT_TRAIN[b'labels']=np.append(DICT_TRAIN[b'labels'], fifth[b'labels'],0)
DICT_TEST = unpickle("cifar-10-batches-py/test_batch")

In [5]:
import os
import sys

In [6]:
x = DICT_TRAIN[b'data_processed']
y = DICT_TRAIN[b'labels']
x_test = DICT_TEST[b'data_processed']
y_test = DICT_TEST[b'labels']


In [ ]:
# arr = x_test

In [ ]:
# from PIL import Image
# for i in range(0, len(arr)):
#     img = Image.fromarray(arr[i])
#     filename = str(i)+".png"
#     img.save(filename)

In [7]:
def load_image(f):
    gray = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    return gray

In [8]:
from skimage.feature import hog

# HOG / image params
ORIENTATIONS = 9
PIXELS_PER_CELL = (8, 8)
CELLS_PER_BLOCK = (2, 2)

def extract_hog_features(image_gray):
    features, hog_maps = hog(
        image_gray,
        orientations=ORIENTATIONS,
        pixels_per_cell=PIXELS_PER_CELL,
        cells_per_block=CELLS_PER_BLOCK,
        block_norm='L2-Hys',
        visualize=True,
        feature_vector=True
    )
    return features, hog_maps

In [9]:
from skimage.feature import local_binary_pattern

In [10]:
IMAGE_SIZE = (32,32)
X_test = []
hog_img =[]

for f in x_test:
    
    img_crop = load_image(f)
    if img_crop is None or img_crop.size == 0 or img_crop.shape[0] < 4 or img_crop.shape[1] < 4:
        img_crop = f  # fallback
    
    roi_resized = cv2.resize(img_crop, IMAGE_SIZE, interpolation=cv2.INTER_AREA) # type: ignore
    hog_feature, hog_maps = extract_hog_features(roi_resized)

    # ---------- LBP ----------
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(img_crop, n_points, radius, method="uniform")

    # Histogram of LBP
    (lbphist, _) = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2),
        density=True
    )
    # number of bins (common: 16, 32, 64)
    bins = 32

    # compute histogram for each channel
    hist_r = cv2.calcHist([f], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([f], [1], None, [bins], [0, 256])
    hist_b = cv2.calcHist([f], [2], None, [bins], [0, 256])

    # normalize (important!)
    hist_r = cv2.normalize(hist_r, hist_r).flatten()
    hist_g = cv2.normalize(hist_g, hist_g).flatten()
    hist_b = cv2.normalize(hist_b, hist_b).flatten()

    if hog_feature is None:
        continue
    
    hog_img.append(hog_maps)
    X_test.append(np.hstack([hist_r,hist_g,hist_b,hog_feature, lbphist]))

X_test = np.array(X_test)


In [11]:
# IMAGE_SIZE = (64,64)
X = []
hog_img =[]

for f in x:
    img_crop = load_image(f)
    if img_crop is None or img_crop.size == 0 or img_crop.shape[0] < 4 or img_crop.shape[1] < 4:
        img_crop = f  # fallback

    roi_resized = cv2.resize(img_crop, IMAGE_SIZE, interpolation=cv2.INTER_AREA) # type: ignore
    hog_feature, hog_maps = extract_hog_features(roi_resized)
    # ---------- LBP ----------
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(img_crop, n_points, radius, method="uniform")

    # Histogram of LBP
    (lbphist, _) = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2),
        density=True
    )

    # number of bins (common: 16, 32, 64)
    bins = 32

    # compute histogram for each channel
    hist_r = cv2.calcHist([f], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([f], [1], None, [bins], [0, 256])
    hist_b = cv2.calcHist([f], [2], None, [bins], [0, 256])

    # normalize (important!)
    hist_r = cv2.normalize(hist_r, hist_r).flatten()
    hist_g = cv2.normalize(hist_g, hist_g).flatten()
    hist_b = cv2.normalize(hist_b, hist_b).flatten()

    if hog_feature is None:
        continue
    
    hog_img.append(hog_maps)
    X.append(np.hstack([hist_r,hist_g,hist_b,hog_feature, lbphist]))

X = np.array(X)
# y = np.array(y)

# print(f"[INFO] Total sampel fitur: {X.shape[0]}, dimensi fitur: {X.shape[1]}")

In [12]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
import sklearn.ensemble as en

In [14]:
x

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [15]:
y

array([6, 9, 9, ..., 9, 1, 1], shape=(50000,))

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y ,
                                   random_state=42, 
                                   test_size=0.001, 
                                   shuffle=True)

In [17]:
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [18]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [20]:
# # Creating a support vector classifier
# model = make_pipeline(
#     RobustScaler(),
#     SVC(random_state=42, tol=1e-5, class_weight="balanced",max_iter=500, C=1),
# )
from sklearn.feature_selection import RFE
model = xgb.XGBClassifier()
# model = SVC(C=1, kernel="linear")

rfe = RFE(estimator=model, n_features_to_select=300, step=0.5)

In [21]:
# clf.fit(X_train, y_train)

In [22]:
model.fit(X_train, y_train)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


~323 features

In [23]:
result_xgb = model.predict(X_test)

In [24]:
result_xgb

array([3, 8, 1, ..., 5, 1, 7], shape=(10000,))

In [25]:
# accuracy_score(dict_test['y'], result_xgb)
accuracy_score(y_test, result_xgb)

0.6348

hog

60 features -> 0.4997

200 features -> 0.5764

323 features -> 0.587

concat lbp hog
0.604

concat color too
0.6348 all 430

200 features 0.6167
300 features 0.6296

SVM

0.4871



In [26]:
from sklearn.metrics import classification_report

In [27]:
# print(classification_report(dict_test['y'], result_xgb))
print(classification_report(y_test, result_xgb))

              precision    recall  f1-score   support

           0       0.68      0.70      0.69      1000
           1       0.75      0.74      0.74      1000
           2       0.57      0.52      0.54      1000
           3       0.45      0.44      0.44      1000
           4       0.56      0.58      0.57      1000
           5       0.53      0.55      0.54      1000
           6       0.72      0.73      0.72      1000
           7       0.69      0.66      0.68      1000
           8       0.69      0.71      0.70      1000
           9       0.71      0.72      0.72      1000

    accuracy                           0.63     10000
   macro avg       0.63      0.63      0.63     10000
weighted avg       0.63      0.63      0.63     10000



In [28]:
model.predict(X_test[:1])

array([3])

In [29]:
y_test[:1]

[3]

In [30]:
DICT_TEST.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames', b'data_processed'])

label_namesq]q(UairplaneqU
automobileqUbirdqUcatqUdeerq	Udogq
UfrogqUhorseqUshipq
Utruckq

In [31]:
from joblib import dump, load
dump(model, "xgb.joblib")

['xgb.joblib']

In [32]:
model_loaded = load("xgb.joblib")

In [33]:
model_loaded.predict(X_test[:1])

array([3])

In [34]:
def read_meta_text(filepath):
    metadata = {}
    with open(filepath, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                metadata[key.strip()] = value.strip()
    return metadata

In [35]:
read_meta_text("cifar-10-batches-py/batches.meta")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte